# Modelo de detecção de sinais de depressão

In [1]:
#Importes

#Funções próprias
import funcs
from funcs.clean_text import clean_text
from funcs.label_2_embedding import label2Embedding

#Visualização de dados
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score

#Pré-processamento
!pip install emot
import re
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

#Manipulação de dados
import nltk
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
import json
from sklearn.model_selection import train_test_split
import numpy as np

#Para uso do GloVe
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

#Redes neurais
import torch
from torch import nn

import time
from IPython.display import clear_output
import torch.nn.functional as F

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eduarda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-1-780a2463368c>:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
CUDA_flag = torch.cuda.is_available() # a flag to check if CUDA is available for GPU use
CUDA_flag

C:\Users\eduarda\anaconda3\envs\data_science\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 8000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [3]:
np.random.seed(666)
torch.manual_seed(666)

In [4]:
dataset = pd.read_csv('data\Dataset_Depressao_24_11_2020.csv', index_col=0)
dataset.head()

,usuario,texto,depressao,texto_traduzido
0,lIlIIIlllllIII,arrombada sem mae querendo cancelar um esquizofrênico\r\n\r\ndeu de internet por hoje https://t.co/I8SzMHuYV4,1,broken into without a mother wanting to cancel a schizophrenic\r\n\r\ngave internet for today https://t.co/I8SzMHuYV4
1,rnnpgn,Não tem nada absolutamente mais SEM GRAÇA do que essa galera comentando em perfis esportivos com a foto do Casagrande,0,There is nothing absolutely NO MORE FREE than this crowd commenting on sports profiles with Casagrande's photo
2,KMoraez,Abalada com toda a situação da minha vida,0,Shaken by the whole situation of my life
3,rammassss,aí sempre fico feliz pelas conquistas dos meus amigos sabe.... sério,1,then I am always happy for the achievements of my friends you know .... seriously
4,fuckjxxnny_oh,respirando fundo pra nao socar a cara de ngm,1,taking a deep breath not to punch anyone in the face


In [5]:
dataset['texto_traduzido'].dtype

dtype('O')

In [6]:
dataset['texto_traduzido'] = dataset['texto_traduzido'].astype(str)

# Pré-processamento

## "Tradução" de emojis

In [7]:
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",","").split()))
        text = text.replace(":"," ")
    return text

In [8]:
dataset['documento_processado'] = dataset['texto_traduzido'].apply(lambda x: convert_emojis(x))

In [9]:
dataset.head()

,usuario,texto,depressao,texto_traduzido,documento_processado
0,lIlIIIlllllIII,arrombada sem mae querendo cancelar um esquizofrênico\r\n\r\ndeu de internet por hoje https://t.co/I8SzMHuYV4,1,broken into without a mother wanting to cancel a schizophrenic\r\n\r\ngave internet for today https://t.co/I8SzMHuYV4,broken into without a mother wanting to cancel a schizophrenic\r\n\r\ngave internet for today https //t.co/I8SzMHuYV4
1,rnnpgn,Não tem nada absolutamente mais SEM GRAÇA do que essa galera comentando em perfis esportivos com a foto do Casagrande,0,There is nothing absolutely NO MORE FREE than this crowd commenting on sports profiles with Casagrande's photo,There is nothing absolutely NO MORE FREE than this crowd commenting on sports profiles with Casagrande's photo
2,KMoraez,Abalada com toda a situação da minha vida,0,Shaken by the whole situation of my life,Shaken by the whole situation of my life
3,rammassss,aí sempre fico feliz pelas conquistas dos meus amigos sabe.... sério,1,then I am always happy for the achievements of my friends you know .... seriously,then I am always happy for the achievements of my friends you know .... seriously
4,fuckjxxnny_oh,respirando fundo pra nao socar a cara de ngm,1,taking a deep breath not to punch anyone in the face,taking a deep breath not to punch anyone in the face


##Limpeza de texto

In [10]:
dataset["documento_processado"] = [tknzr.tokenize(word) for word in dataset["documento_processado"]]

In [11]:
dataset["documento_processado"] = [clean_text(sentence) for sentence in dataset["documento_processado"]]

In [12]:
dataset.head()

,usuario,texto,depressao,texto_traduzido,documento_processado
0,lIlIIIlllllIII,arrombada sem mae querendo cancelar um esquizofrênico\r\n\r\ndeu de internet por hoje https://t.co/I8SzMHuYV4,1,broken into without a mother wanting to cancel a schizophrenic\r\n\r\ngave internet for today https://t.co/I8SzMHuYV4,"[broken, without, mother, wanting, cancel, schizophrenic, gave, internet, today, https, t.co/i8szmhuyv4]"
1,rnnpgn,Não tem nada absolutamente mais SEM GRAÇA do que essa galera comentando em perfis esportivos com a foto do Casagrande,0,There is nothing absolutely NO MORE FREE than this crowd commenting on sports profiles with Casagrande's photo,"[nothing, absolutely, free, crowd, commenting, sports, profiles, casagrande's, photo]"
2,KMoraez,Abalada com toda a situação da minha vida,0,Shaken by the whole situation of my life,"[shaken, whole, situation, life]"
3,rammassss,aí sempre fico feliz pelas conquistas dos meus amigos sabe.... sério,1,then I am always happy for the achievements of my friends you know .... seriously,"[always, happy, achievements, friends, know, ..., seriously]"
4,fuckjxxnny_oh,respirando fundo pra nao socar a cara de ngm,1,taking a deep breath not to punch anyone in the face,"[taking, deep, breath, punch, anyone, face]"


## Codificando o texto para o nosso modelo
Agora iremos fazer isso utilizando o GloVe

In [13]:
cwd = os.getcwd()
cwd

'c:\\Users\\eduarda\\Documents\\Workspace\\project\\2020_INF425_NLP_SeReS\\eda'

In [14]:
num_dims = 50

glove_file = datapath(cwd+f'/data/glove.6B.{num_dims}d.txt')
tmp_file   = get_tmpfile(cwd+f"/data/glove.6B.{num_dims}d_word2vec.txt")
_          = glove2word2vec(glove_file, tmp_file)

filename_txt = cwd+f"/data/glove.6B.{num_dims}d_word2vec.txt"
modelo = KeyedVectors.load_word2vec_format(filename_txt)

In [15]:
max_len = 280        # comprimento máximo da mensagem (em número de palavras)
encoded_docs = []    # inicializa a lista de documentos codificados

for sentence in dataset['documento_processado']: # para cada token
  encoded_d = [label2Embedding(t, modelo) for t in sentence]
  encoded_d = [vec.tolist() for vec in encoded_d if vec is not None]

  # adiciona o padding, se necessário
  padding_word_vecs = [np.zeros(num_dims).tolist()]*max(0, max_len-len(encoded_d)) 
  encoded_d = padding_word_vecs + encoded_d
  
  # trunca o documento e salva na lista de documentos codificados
  encoded_docs.append(encoded_d[:max_len]) 


encoded_docs_arrays = [np.vstack(sentence) for sentence in encoded_docs]

In [16]:
type(encoded_docs_arrays), type(encoded_docs_arrays[0]), encoded_docs_arrays[0].shape

(list, numpy.ndarray, (280, 50))

In [17]:
dataset['documento_processado'] = pd.Series(encoded_docs_arrays)
dataset['documento_processado'][0]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 5.71210027e-01, -2.34229997e-01,  1.50670004e+00, ...,
        -7.17770010e-02,  4.30299997e-01,  1.89400002e-01],
       [ 2.77510000e-04,  4.26730007e-01, -8.29380006e-02, ...,
         3.96149993e-01,  1.88570004e-02,  1.75359994e-01],
       [ 1.26170003e+00,  2.66339988e-01,  4.76870000e-01, ...,
         3.15109998e-01,  6.34980023e-01, -6.56949997e-01]])

In [18]:
dataset['documento_processado'].shape, dataset['documento_processado'][0].shape, dataset['documento_processado'][0].dtype

((1411,), (280, 50), dtype('float64'))

## Usando o modelo de classificação de emoções

In [19]:
from funcs.model_class import *

In [20]:
Model = torch.load(cwd+'/../models/emotions_classifier_LSTM.pth', map_location=torch.device('cpu'))

In [21]:
Model.load_state_dict(torch.load(cwd+'/../dicts/emotions_classifier_dict_LSTM', map_location=torch.device('cpu')))
Model.eval()

moodClassifierWithLasersOnSteroids(
  (embedding): Sequential(
    (0): LSTM(100, 50, batch_first=True)
  )
  (ann): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=50, out_features=6, bias=True)
  )
  (soft): Sequential(
    (0): LogSoftmax(dim=1)
  )
)

In [23]:
e = next(Model.embedding[0].parameters())
e.data

tensor([[ 0.0099,  0.0583, -0.1418,  ..., -0.0183, -0.0139,  0.0427],
        [-0.0857,  0.0209, -0.0693,  ..., -0.0318, -0.0408,  0.1099],
        [-0.1435, -0.0190,  0.3701,  ..., -0.0698, -0.0069, -0.1877],
        ...,
        [ 0.0467, -0.0495,  0.0721,  ..., -0.1968,  0.1171, -0.2641],
        [ 0.4760, -0.0488,  0.1691,  ..., -0.1061, -0.1291, -0.3212],
        [ 0.2292,  0.1495,  0.2034,  ..., -0.2226,  0.6667,  0.0490]])

In [24]:
X_test = np.dstack(dataset['documento_processado'].values).transpose(2,0,1)
X_text = torch.FloatTensor(X_test)
X_test.shape

(1411, 280, 50)

In [27]:
preds = Model.forward(X_test)

TypeError: 'int' object is not callable

In [ ]:
dataset['documento_processado'][0]

In [ ]:
model.forward(dataset['documento_processado'][0])

In [ ]:
dataset["previsoes"] = [model.forward(x) for x in dataset['documento_processado']]
dataset.head()

## Preparando o input (Aka a nova distribuição média de emoções)

Devemos pegar as softmax das emoções de vários tweets de um mesmo usuário e transformar em um só para o nosso modelo final

In [ ]:
idx2word = list(word2idx.keys()) # apenas transforma as chaves (palavras ordenadas) do dicionário word2idx em uma lista

# testando a conversão "index to word":
print(f'word for index 0:    {idx2word[0]}')
print(f'word for index 100": {idx2word[100]}')

In [ ]:
max_len = 30         # comprimento máximo da mensagem (em número de palavras)
encoded_docs = []    # inicializa a lista de documentos codificados

for doc in dataset['documento_processado']: # para cada texto
  encoded_d = [word2idx.get(t,word2idx['<OOV>']) for t in doc]    # codifica o documento usando o dicionário word2idx
  encoded_d += [word2idx['<PAD>']]*max(0, max_len-len(encoded_d))    # adiciona o padding, se necessário
  
  encoded_docs.append(encoded_d[:max_len])                           # trunca o documento e salva na lista de documentos codificados

len(encoded_docs)  

In [ ]:
dataset['documento_processado'] = encoded_docs
dataset.head()

#Chamando o modelo de previsão de emoções

In [ ]:
import myclass
from myclass import *

In [ ]:
model = torch.load('/content/drive/My Drive/Análise de Sentimentos/Projeto para 08-09/Models/newmodel.pth')

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/Análise de Sentimentos/Projeto para 08-09/Models/newmodel_dict'))
model.eval()

In [ ]:
e = next(model.embedding[0].parameters())
e.data

#Adcionando a *feature* do modelo de previsão de emoções

In [ ]:
X_test = np.vstack(dataset['documento_processado'])
X_test = torch.FloatTensor(X_test)
X_test.shape

In [ ]:
preds = model.forward(X_test)

In [ ]:
dataset

## Criando colunas com os preds

In [ ]:
dataset["previsoes"] = [torch.exp(pred).detach().numpy() for pred in preds]
dataset.head()

In [ ]:
dataset.query("depressao==1")

# Tirando a média

In [ ]:
df = dataset.copy()
dataset.set_index('usuario', inplace=True)

In [ ]:
#Funções necessárias

def get_matriz_user(distributions):
  """
  Pega as distruibuições do usuário (O resultado o modelo de emoções de todos os tweets) e cria uma matriz
  """
  matriz = np.vstack(distributions)

  return matriz


def get_media_user(matriz):
  """
  Pega a matriz dos vetores resultantes do modelo de previsão de emoções e faz a média
  """
  media = matriz.mean(axis=0).reshape(1,-1)

  return media

In [ ]:
unique_users1 = df.query('depressao == 1')['usuario'].unique()
unique_users0 = df.query('depressao == 0')['usuario'].unique()

df_final = pd.DataFrame(columns=['user','emotions_avg_dist', 'depression'])

#EL GAMBIARRA (arrumar depois)
  
for user in unique_users1:
  df_user = dataset.loc[user] #Pegando o usuário e seus tweets
  distributions_of_user = df_user['previsoes'] #Pegando as distribuições

  matriz_de_distribuicao = get_matriz_user(distributions_of_user)
  media_usuario = get_media_user(matriz_de_distribuicao)
  media_usuario = media_usuario/media_usuario.sum()

  df_of_new_row = pd.DataFrame(data=[[user, media_usuario, 1]], columns=['user','emotions_avg_dist','depression']) #Criando uma nova linha
  df_final = df_final.append(df_of_new_row,ignore_index=True) #Adcionando a nova linha no dataframe

for user in unique_users0:
  df_user = dataset.loc[user] #Pegando o usuário e seus tweets
  distributions_of_user = df_user['previsoes'] #Pegando as distribuições

  matriz_de_distribuicao = get_matriz_user(distributions_of_user)
  media_usuario = get_media_user(matriz_de_distribuicao)
  media_usuario = media_usuario/media_usuario.sum()

  df_of_new_row = pd.DataFrame(data=[[user, media_usuario, 0]], columns=['user','emotions_avg_dist','depression']) #Criando uma nova linha
  df_final = df_final.append(df_of_new_row,ignore_index=True) #Adcionando a nova linha no dataframe

df_final

In [ ]:
df_final.iloc[10]['emotions_avg_dist'].sum()

In [ ]:
df_final.query('depression==1')

In [ ]:
df_final = df_final.sample(frac=1).reset_index(drop=True)
df_final

##Train Test Split

In [ ]:
X = df_final.drop(['depression'], axis=1)
y = df_final['depression']

In [ ]:
X_train_df, X_test_df, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=666, shuffle=True)

In [ ]:
X_train_df.shape, y_train.shape, X_test_df.shape, y_test.shape

In [ ]:
y_train.value_counts()

In [ ]:
y_train.value_counts(normalize=True) * 100

In [ ]:
y_test.value_counts(normalize=True) * 100

In [ ]:
y_test.value_counts()

In [ ]:
type(y_test.to_numpy())

## Criando ***El Modelo***

In [ ]:
class depression_detection(nn.Module):
  '''Modelo classificador de emoções
  '''
  # ----------------------------------------------#
  # Método construtor
  def __init__(self, n_in, num_layers, num_units): 

    super().__init__()  
    ann_seq       = [] # 

    #--------------------------------------------------------------------------#
    # ANN: Rede Neural Artifical Tradicional, com regressão logística na saída
    for i in range(1, num_layers):
      ann_seq.append(nn.Linear(in_features=num_units[i-1], out_features=num_units[i]))
      ann_seq.append(nn.ReLU(inplace=True))
    ann_seq.append(nn.Linear(in_features=num_units[-1], out_features=1))
    ann_seq.append(nn.Sigmoid())
    
    #--------------------------------------------------------------------------#
    # "merge" de todas as camamadas em uma layer sequencial 
    # (uma sequência para cada etapa)
    self.ann= nn.Sequential(*ann_seq)           # etapa ANN
    #--------------------------------------------------------------------------#

  def forward(self, x): 
    '''
    Processamento realizado ao chamar y=modelo(x)
    '''
    x = self.ann(x)        # passa o embedding médio pelas camadas da ANN
    return x  

In [ ]:
def train_loop(model, data_train, data_test, max_epochs = 1000, print_iters = 5):
  X_train, Y_train = data_train
  X_test, Y_test = data_test
  losses = []
  accs = []
  for i in range(max_epochs): # para cada época

      #-----------------------------------#
      # INÍCIO DO WORKFLOW DO TREINAMENTO #
      # 
      # Add mistura
      Y_pred = model.forward(X_train)         # apresente os dados de entrada para o modelo, e obtenha a previsão    
      loss = criterion(Y_pred, Y_train)                      # calcule a perda (o custo, o erro)
      optimizer.zero_grad()                   # inicialize os gradientes
      loss.backward()                         # backpropagation sobre a perda atual (cálculo dos novos gradientes) 
      optimizer.step()                        # atualização dos parâmetros da rede utilizando a regra do otimizador escolhido

      
      # FIM DO WORKFLOW DO TREINAMENTO    #
      #-----------------------------------#

      # ------ Bloco Opcional ------ #
      # Salvando métricas
      losses.append(loss)                     # salvando a perda atual
      acc = calc_accuracy(Y_pred, Y_train)     # calcula a taxa de acerto atual
      accs.append(acc)
      
      # Imprimindo resultados parciais
      if i % print_iters ==0: # a cada 10 iterações
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}') 
      #-----------------------------------#

  #----------------------------------------------------------------------------# 
  print('\n# Finished training!')
  print(f'# --> epoch: {i}  \n# --> initial loss: {losses[0]:10.8f} ,  \n# --> accuracy: {acc:2.8f} , \n# --> final loss: {losses[-1]:10.8f}')
  
  # retornando resultados
  return model, losses, accs

# Redefinindo cálculo da taxa de acerto 
def calc_accuracy(y_pred, y_true):
  ''' Helper function para calcular a taxa de acerto deste exemplo.
  '''
  num_hits  = torch.sum(torch.round(y_pred)==y_true).numpy()
  num_total =  float(y_true.numel())
  acc=  num_hits/num_total
  return acc

In [ ]:

X_train = np.vstack(X_train_df['emotions_avg_dist'])
Y_train = np.vstack(y_train).reshape(-1,1)
X_train.shape, X_train[0].shape, Y_train.shape, Y_train[0].shape


In [ ]:
X_test = np.vstack(X_test_df['emotions_avg_dist'])
Y_test = np.vstack(y_test).reshape(-1,1)
X_test.shape, X_test[0].shape, Y_test.shape, Y_test[0].shape

In [ ]:
data_train = (torch.FloatTensor(X_train), torch.FloatTensor(Y_train))

data_test = (torch.FloatTensor(X_test), torch.FloatTensor(Y_test))

## Melhorando com hiperparâmetros

In [ ]:
# Código baseado no github do professor Bruno Fontana. 
from sklearn.model_selection import ParameterGrid

Ajustar_Hiperparametros = True     

param_dict = {}
param_dict['learning_rate'] = np.linspace(0.01, 0.1, 10) # [0.1, 0.01, 0.001] # exemplos de buscas: np.linspace(1e-2,5e-2,5) ou np.logspace(-3,-1,20)
param_dict['layers'] = [
                        (2, [6, 40]),
                        (2, [6, 48]),
                        (2, [6, 52]),
                        (3, [6, 48, 8]),
                        (3, [6, 48, 16]),
                        (3, [6, 48, 32]),
                        (3, [6, 48, 48]),
                        (3, [6, 48, 64])]
                        # (4, [6, 100, 50, 25]), 
                        # (5, [6, 200, 100, 50, 25]),
                        # (6, [6, 400, 200, 100, 50, 25])]              # exemplos de buscas: np.arange(start=1,stop=4)
# param_dict['num_units'] = [[6, 100, 50, 25], [6, 200, 100, 50, 25]]         # exemplos de buscas: np.arange(10,15)

param_dict['max_epochs']= [3000]     # maximum number of iterations running over the training set

Num_HyperParams_to_test = len(ParameterGrid(param_dict))

In [ ]:
criterion = nn.BCELoss() # binary cross-entropy

if Ajustar_Hiperparametros:
  acc_test_list = []
  max_acc_test = -np.inf
  best_params = None
  idx_best = None
  for idx, params in enumerate(ParameterGrid(param_dict)):
    learning_rate = params['learning_rate']      # learning rate for parameter updates
    L, num_units = params['layers']                              # depth of ANN (number of hidden layers)
    #num_units = params['num_units']                    # 2**Nu_exp = Number of units per HL
    max_epochs = params['max_epochs']            # num. of iterations over the full dataset

    print(f'Testing parameters {idx}/{Num_HyperParams_to_test}: {params}')

    torch.manual_seed(666)
    model = depression_detection(n_in=6, num_layers=L, num_units=num_units)
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) 
    model, losses, accs = train_loop(model, data_train, data_test, max_epochs=max_epochs, print_iters=1000)

    # print(f'Model {model}',f'Criterion: {criterion}',f'Optimizer: {optimizer}',sep=2*'\n')

    X_test, Y_test = data_test
    with torch.no_grad():
      Y_pred = model(X_test)
      num_hits  = torch.sum(torch.round(Y_pred)==Y_test).numpy()
      num_total =  float(Y_test.numel())
      acc_test = num_hits/num_total
      # print(f'TEST SCORE: Torch loss {acc_test.item():10.6f}  (ANN)')

    if acc_test > max_acc_test:
      max_acc_test = acc_test
      best_params = params
      losses_best, acc_best = (losses, accs)
      idx_best = idx
      best_model = model

    acc_test_list.append(acc_test)


    # clear_output(wait=True)
  print("best_params", best_params, "acc_test_list", acc_test_list, "idx_best", idx_best, "acc_best", acc_test_list[idx_best], sep="\n")

else:
  model = depression_detection(n_in=6, num_layers=2, num_units=[6,48])
  optimizer = torch.optim.Adam(model.parameters(), lr = 0.1) 
  best_model, losses_best, acc_best = train_loop(model, data_train, data_test,max_epochs=2000, print_iters=1)

In [ ]:
print(best_model)

In [ ]:
# Model, losses, accs = train_loop(model, data_train, data_test, max_epochs=1000, print_iters=1) # note que o modelo é sobrescrito pela saída treinada

In [ ]:
X_train = torch.FloatTensor(X_train)

In [ ]:
Y_pred = best_model(X_train)

#Visualizando resultados

##Gráfico de Loss e Accuracy

In [ ]:
def plot_loss_and_accuracy(losses, accs):

  fig, ax_tuple = plt.subplots(1, 2, figsize=(16,6))
  fig.suptitle('Loss and accuracy')

  for i, (y_label, y_values) in enumerate(zip(['BCE loss','Accuracy'],[losses, accs])):
    ax_tuple[i].plot(range(len(y_values)),  y_values, label='train')
    ax_tuple[i].set_xlabel('epochs')
    ax_tuple[i].set_ylabel(y_label)
    ax_tuple[i].legend()
  
  ax_tuple[1].set_ylim([0,1])

In [ ]:
plot_loss_and_accuracy(losses_best, acc_best)

##Confusion Matrix

In [ ]:
X_test = np.vstack(X_test)
Y_test = np.vstack(y_test).reshape(-1,1)
X_test = torch.FloatTensor(X_test)
Y_test =  torch.FloatTensor(Y_test)
X_test.shape, Y_test.shape

In [ ]:
Y_pred = best_model(X_test)
#Y_pred[:10]

In [ ]:
Y_test.dtype

In [ ]:
X, Y = data_test
with torch.no_grad():
    Y2 = best_model(X)
    num_hits  = torch.sum(torch.round(Y2)==Y).numpy()
    num_total =  float(Y.numel())
    acc_test = num_hits/num_total

acc_test

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

sns.set(font_scale=1.5)

# confusion matrix daora TEM Q MELHORAR
def plot_conf_mat(y_test, y_preds, norm=None):
   fig, ax = plt.subplots(figsize=(8, 6))
   ax = sns.heatmap(confusion_matrix(y_test, y_preds, normalize=norm),
                   annot=True,
                    fmt="d",
                    cmap="YlOrRd")
   plt.xlabel("Resultado previsto")
   plt.ylabel("Resultado real")


plot_conf_mat(Y_test.detach().numpy(), Y_pred.detach().numpy().round())

In [ ]:
Y_predtrain = best_model(X_train)
plot_conf_mat(Y_train, Y_predtrain.detach().numpy().round())

## F1 Score

In [ ]:
from sklearn.metrics import f1_score

f1_score(Y_test.detach().numpy(), Y_pred.detach().numpy().round())

## Recall Score

In [ ]:
from sklearn.metrics import recall_score

recall_score(Y_test.detach().numpy(), Y_pred.detach().numpy().round())

##Precision

In [ ]:
from sklearn.metrics import precision_score

precision_score(Y_test.detach().numpy(), Y_pred.detach().numpy().round())

## ROC Curve

In [ ]:
# função pra plotar a ROC curve

def plot_roc_curve(fpr, tpr):
    
    plt.plot(fpr, tpr, color="orange", label="ROC")  
    plt.plot([0,1],[0,1], color="darkblue", linestyle="--", label="Guessing")
    
    plt.xlabel("Taxa de falsos positivos  (fpr)")
    plt.ylabel("Taxa verdadeira positiva (tpr)")
    plt.title("Receiver Operating Characteristics (ROC) curve")
    plt.legend()
    plt.show()

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(Y_test.detach().numpy(), Y_pred.detach().numpy())
plot_roc_curve(fpr, tpr)

## ROC/AUC Score

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(Y_test.detach().numpy(), Y_pred.detach().numpy())

## Classification Report

In [ ]:
from sklearn.metrics import classification_report

# compare the truth labels to the predicted labels
print(classification_report(Y_test.detach().numpy(), Y_pred.detach().numpy().round()))

##Falsos Negativos

In [ ]:
moods = {"Anger": 0, "Fear" : 1, "Joy" : 2, "Love" : 3, "Sadness" : 4, "Suprise" : 5}
sdoom = {0 : "Anger", 1: "Fear", 2: "Joy", 3: "Love", 4: "Sadness", 5: "Suprise"}
emotions = list(moods.keys())

In [ ]:
Ytrue1 = Y_test.detach().numpy() == 1
Ypred0 = Y_pred.detach().numpy().round() == 0

In [ ]:
X_test_df[Ytrue1.reshape(-1) & Ypred0.reshape(-1)]  #Onde era verdadeiro e foi previsto falso  (FALSO NEGATIVO)

Primeiro exemplo

In [ ]:
usuario_teste = 'je_ecobaby'
dataset.loc[usuario_teste][['texto', 'depressao','previsoes']]

In [ ]:
dataset.loc[usuario_teste]['previsoes'].apply(lambda x: sdoom[np.argmax(np.array(x))])

In [ ]:
plt.figure(figsize=(10, 6))
jairo = dataset.set_index('user').loc[usuario_teste][0]
jairo = jairo.reshape(-1)
plt.bar(emotions, jairo, color=['lightcoral', 'khaki', 'bisque', 'lightsteelblue', 'lightgreen', 'thistle'])
plt.show()

In [ ]:
jairo = X_test_df.set_index('user').loc[usuario_teste][0]
jairo = jairo.reshape(-1)
sns.barplot(y=jairo,x=emotions)

##Falsos Positive

In [ ]:
Ytrue0 = Y_test.detach().numpy() == 0
Ypred1 = Y_pred.detach().numpy().round() == 1

In [ ]:
X_test_df[Ytrue0.reshape(-1) & Ypred1.reshape(-1)]  #Onde era falso e foi previsto verdadeiro  (FALSO POSITIVE)

Primeiro exemplo

In [ ]:
usuario_teste = 'litarib'
dataset.loc[usuario_teste][['texto', 'depressao','previsoes']]

In [ ]:
dataset.loc[usuario_teste]['previsoes'].apply(lambda x: sdoom[np.argmax(np.array(x))])

In [ ]:
plt.figure(figsize=(10, 6))
jairo = X_test_df.set_index('user').loc[usuario_teste][0]
jairo = jairo.reshape(-1)
plt.bar(emotions, jairo, color=['lightcoral', 'khaki', 'bisque', 'lightsteelblue', 'lightgreen', 'thistle'])
plt.show()

In [ ]:
jairo = X_test_df.set_index('user').loc[usuario_teste][0]
jairo = jairo.reshape(-1)
sns.barplot(y=jairo,x=emotions)

##True Positive

In [ ]:
Ytrue1 = Y_test.detach().numpy() == 1
Ypred1 = Y_pred.detach().numpy().round() == 1

In [ ]:
X_test_df[Ytrue1.reshape(-1) & Ypred1.reshape(-1)]  #Onde era verdadeiro e foi previsto verdadeiro  (True positive)

In [ ]:
usuario_teste = 'ggukpersona'

In [ ]:
dataset.loc[usuario_teste][['texto', 'depressao','previsoes']]

In [ ]:
dataset.loc[usuario_teste]['previsoes'].apply(lambda x: sdoom[np.argmax(np.array(x))])

In [ ]:
plt.figure(figsize=(10, 6))
jairo = X_test_df.set_index('user').loc[usuario_teste][0]
jairo = jairo.reshape(-1)
plt.bar(emotions, jairo, color=['lightcoral', 'khaki', 'bisque', 'lightsteelblue', 'lightgreen', 'thistle'])
plt.show()

In [ ]:
jairo = X_test_df.set_index('user').loc[usuario_teste][0]
jairo = jairo.reshape(-1)
sns.barplot(y=jairo,x=emotions)

Segundo exemplo

In [ ]:
usuario_teste = 'barbaraton'

In [ ]:
dataset.loc[usuario_teste][['texto', 'depressao','previsoes']]

In [ ]:
dataset.loc[usuario_teste]['previsoes'].apply(lambda x: sdoom[np.argmax(np.array(x))])

In [ ]:
plt.figure(figsize=(10, 6))
jairo = X_test_df.set_index('user').loc[usuario_teste][0]
jairo = jairo.reshape(-1)
plt.bar(emotions, jairo, color=['lightcoral', 'khaki', 'bisque', 'lightsteelblue', 'lightgreen', 'thistle'])
plt.show()

In [ ]:
jairo = X_test_df.set_index('user').loc[usuario_teste][0]
jairo = jairo.reshape(-1)
sns.barplot(y=jairo,x=emotions)